##### XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [6]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [7]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [8]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:
document = ET.parse( './data/mondial_database.xml' )

In [112]:
import operator
data = {}
for child in document.iterfind('country'):
    name = child.find('name').text
    if child.find('infant_mortality') != None:
        infant_mortality = float(child.find('infant_mortality').text)
    data[infant_mortality] = name

sorted(data.items())

[(1.81, 'Monaco'),
 (2.13, 'Japan'),
 (2.48, 'Bermuda'),
 (2.53, 'Singapore'),
 (2.6, 'Sweden'),
 (2.63, 'Czech Republic'),
 (2.73, 'Hong Kong'),
 (3.13, 'Macao'),
 (3.15, 'Iceland'),
 (3.31, 'Italy'),
 (3.33, 'Spain'),
 (3.36, 'Finland'),
 (3.4, 'Anguilla'),
 (3.46, 'Germany'),
 (3.47, 'Melilla'),
 (3.59, 'Malta'),
 (3.64, 'Belarus'),
 (3.66, 'Netherlands'),
 (3.69, 'Andorra'),
 (3.73, 'Switzerland'),
 (3.74, 'Ireland'),
 (3.86, 'Jersey'),
 (3.93, 'South Korea'),
 (3.98, 'Israel'),
 (4.04, 'Slovenia'),
 (4.1, 'Denmark'),
 (4.16, 'Austria'),
 (4.17, 'Isle of Man'),
 (4.18, 'Belgium'),
 (4.28, 'Luxembourg'),
 (4.33, 'Liechtenstein'),
 (4.43, 'Australia'),
 (4.44, 'United Kingdom'),
 (4.48, 'Svalbard'),
 (4.49, 'Wallis and Futuna'),
 (4.52, 'San Marino'),
 (4.59, 'Norfolk Island'),
 (4.7, 'Cuba'),
 (4.71, 'Canada'),
 (4.78, 'French Polynesia'),
 (5.09, 'Hungary'),
 (5.35, 'Slovakia'),
 (5.46, 'New Caledonia'),
 (5.5, 'Northern Mariana Islands'),
 (5.51, 'Guam'),
 (5.71, 'Faroe Islands'),

<Element 'country' at 0xb37cb2ec>